导入所需依赖：

In [1]:
import mindspore as ms
import mindspore.nn as nn
import mindspore.dataset as ds
import mindspore.dataset.transforms.c_transforms as c_transforms
import mindspore.dataset.vision.c_transforms as c_vision
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.common.tensor import Tensor
from mindspore.numpy import argmax
from mindvision.engine.callback import LossMonitor

import math
import cv2
import os
import numpy as np
from scipy.stats import truncnorm

设置动态图模式和使用设备：

In [2]:
ms.set_context(mode=ms.GRAPH_MODE, device_target="Ascend")

# 训练

定义超参及路径

In [3]:
train_dir = "dataset/train"
epochs = 20
lr = 0.01
momentum = 0.9

定义创建数据集类

In [4]:
def create_dataset(dir, batch_size, repeat_num):
    """定义数据集"""
    dataset = ds.ImageFolderDataset(dir, shuffle=True)

    image_size = [224, 224]
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]
    trans = [
        c_vision.Decode(),
        c_vision.Resize(image_size),
        c_vision.Normalize(mean=mean, std=std),
        c_vision.HWC2CHW()
    ]

    # 实现数据的map映射、批量处理和数据重复的操作
    type_cast_op = c_transforms.TypeCast(ms.int32)
    dataset = dataset.map(operations=trans, input_columns="image")
    dataset = dataset.map(operations=type_cast_op, input_columns="label")
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.repeat(repeat_num)

    return dataset

构建ResNet50网络基础元素

In [5]:
def conv_variance_scaling_initializer(in_channel, out_channel, kernel_size):
    fan_in = in_channel * kernel_size * kernel_size
    scale = 1.0
    scale /= max(1., fan_in)
    stddev = (scale ** 0.5) / .87962566103423978
    mu, sigma = 0, stddev
    weight = truncnorm(-2, 2, loc=mu, scale=sigma).rvs(out_channel * in_channel * kernel_size * kernel_size)
    weight = np.reshape(weight, (out_channel, in_channel, kernel_size, kernel_size))
    return Tensor(weight, dtype=ms.float32)

In [6]:
def _weight_variable(shape, factor=0.01):
    init_value = np.random.randn(*shape).astype(np.float32) * factor
    return Tensor(init_value)

In [7]:
def calculate_gain(nonlinearity, param=None):
    """calculate_gain"""
    linear_fns = ['linear', 'conv1d', 'conv2d', 'conv3d', 'conv_transpose1d', 'conv_transpose2d', 'conv_transpose3d']
    res = 0
    if nonlinearity in linear_fns or nonlinearity == 'sigmoid':
        res = 1
    elif nonlinearity == 'tanh':
        res = 5.0 / 3
    elif nonlinearity == 'relu':
        res = math.sqrt(2.0)
    elif nonlinearity == 'leaky_relu':
        if param is None:
            neg_slope = 0.01
        elif not isinstance(param, bool) and isinstance(param, int) or isinstance(param, float):
            neg_slope = param
        else:
            raise ValueError("neg_slope {} not a valid number".format(param))
        res = math.sqrt(2.0 / (1 + neg_slope ** 2))
    else:
        raise ValueError("Unsupported nonlinearity {}".format(nonlinearity))
    return res

In [8]:
def _calculate_fan_in_and_fan_out(tensor):
    """_calculate_fan_in_and_fan_out"""
    dimensions = len(tensor)
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")
    if dimensions == 2:  # Linear
        fan_in = tensor[1]
        fan_out = tensor[0]
    else:
        num_input_fmaps = tensor[1]
        num_output_fmaps = tensor[0]
        receptive_field_size = 1
        if dimensions > 2:
            receptive_field_size = tensor[2] * tensor[3]
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size
    return fan_in, fan_out

In [9]:
def _calculate_correct_fan(tensor, mode):
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out']
    if mode not in valid_modes:
        raise ValueError("Unsupported mode {}, please use one of {}".format(mode, valid_modes))
    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    return fan_in if mode == 'fan_in' else fan_out

In [12]:
def kaiming_normal(inputs_shape, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    fan = _calculate_correct_fan(inputs_shape, mode)
    gain = calculate_gain(nonlinearity, a)
    std = gain / math.sqrt(fan)
    return np.random.normal(0, std, size=inputs_shape).astype(np.float32)

In [11]:
def kaiming_uniform(inputs_shape, a=0., mode='fan_in', nonlinearity='leaky_relu'):
    fan = _calculate_correct_fan(inputs_shape, mode)
    gain = calculate_gain(nonlinearity, a)
    std = gain / math.sqrt(fan)
    bound = math.sqrt(3.0) * std  # Calculate uniform bounds from standard deviation
    return np.random.uniform(-bound, bound, size=inputs_shape).astype(np.float32)

In [13]:
def _conv3x3(in_channel, out_channel, stride=1, use_se=False, res_base=False):
    if use_se:
        weight = conv_variance_scaling_initializer(in_channel, out_channel, kernel_size=3)
    else:
        weight_shape = (out_channel, in_channel, 3, 3)
        weight = Tensor(kaiming_normal(weight_shape, mode="fan_out", nonlinearity='relu'))
    if res_base:
        return nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride,
                         padding=1, pad_mode='pad', weight_init=weight)
    return nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride,
                     padding=0, pad_mode='same', weight_init=weight)

In [14]:
def _conv1x1(in_channel, out_channel, stride=1, use_se=False, res_base=False):
    if use_se:
        weight = conv_variance_scaling_initializer(in_channel, out_channel, kernel_size=1)
    else:
        weight_shape = (out_channel, in_channel, 1, 1)
        weight = Tensor(kaiming_normal(weight_shape, mode="fan_out", nonlinearity='relu'))
    if res_base:
        return nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride,
                         padding=0, pad_mode='pad', weight_init=weight)
    return nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride,
                     padding=0, pad_mode='same', weight_init=weight)

In [15]:
def _conv7x7(in_channel, out_channel, stride=1, use_se=False, res_base=False):
    if use_se:
        weight = conv_variance_scaling_initializer(in_channel, out_channel, kernel_size=7)
    else:
        weight_shape = (out_channel, in_channel, 7, 7)
        weight = Tensor(kaiming_normal(weight_shape, mode="fan_out", nonlinearity='relu'))
    if res_base:
        return nn.Conv2d(in_channel, out_channel,
                         kernel_size=7, stride=stride, padding=3, pad_mode='pad', weight_init=weight)
    return nn.Conv2d(in_channel, out_channel,
                     kernel_size=7, stride=stride, padding=0, pad_mode='same', weight_init=weight)

In [16]:
def _bn(channel, res_base=False):
    if res_base:
        return nn.BatchNorm2d(channel, eps=1e-5, momentum=0.1,
                              gamma_init=1, beta_init=0, moving_mean_init=0, moving_var_init=1)
    return nn.BatchNorm2d(channel, eps=1e-4, momentum=0.9,
                          gamma_init=1, beta_init=0, moving_mean_init=0, moving_var_init=1)


def _bn_last(channel):
    return nn.BatchNorm2d(channel, eps=1e-4, momentum=0.9,
                          gamma_init=0, beta_init=0, moving_mean_init=0, moving_var_init=1)

In [17]:
def _fc(in_channel, out_channel, use_se=False):
    if use_se:
        weight = np.random.normal(loc=0, scale=0.01, size=out_channel * in_channel)
        weight = Tensor(np.reshape(weight, (out_channel, in_channel)), dtype=ms.float32)
    else:
        weight_shape = (out_channel, in_channel)
        weight = Tensor(kaiming_uniform(weight_shape, a=math.sqrt(5)))
    return nn.Dense(in_channel, out_channel, has_bias=True, weight_init=weight, bias_init=0)

构建残差块：

In [18]:
class ResidualBlock(nn.Cell):
    expansion = 4

    def __init__(self,
                 in_channel,
                 out_channel,
                 stride=1,
                 use_se=False, se_block=False):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.use_se = use_se
        self.se_block = se_block
        channel = out_channel // self.expansion
        self.conv1 = _conv1x1(in_channel, channel, stride=1, use_se=self.use_se)
        self.bn1 = _bn(channel)
        if self.use_se and self.stride != 1:
            self.e2 = nn.SequentialCell([_conv3x3(channel, channel, stride=1, use_se=True), _bn(channel),
                                         nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2, pad_mode='same')])
        else:
            self.conv2 = _conv3x3(channel, channel, stride=stride, use_se=self.use_se)
            self.bn2 = _bn(channel)

        self.conv3 = _conv1x1(channel, out_channel, stride=1, use_se=self.use_se)
        self.bn3 = _bn(out_channel)
        if self.se_block:
            self.se_global_pool = P.ReduceMean(keep_dims=False)
            self.se_dense_0 = _fc(out_channel, int(out_channel / 4), use_se=self.use_se)
            self.se_dense_1 = _fc(int(out_channel / 4), out_channel, use_se=self.use_se)
            self.se_sigmoid = nn.Sigmoid()
            self.se_mul = P.Mul()
        self.relu = nn.ReLU()

        self.down_sample = False

        if stride != 1 or in_channel != out_channel:
            self.down_sample = True
        self.down_sample_layer = None

        if self.down_sample:
            if self.use_se:
                if stride == 1:
                    self.down_sample_layer = nn.SequentialCell([_conv1x1(in_channel, out_channel,
                                                                         stride, use_se=self.use_se), _bn(out_channel)])
                else:
                    self.down_sample_layer = nn.SequentialCell([nn.MaxPool2d(kernel_size=2, stride=2, pad_mode='same'),
                                                                _conv1x1(in_channel, out_channel, 1,
                                                                         use_se=self.use_se), _bn(out_channel)])
            else:
                self.down_sample_layer = nn.SequentialCell([_conv1x1(in_channel, out_channel, stride,
                                                                     use_se=self.use_se), _bn(out_channel)])

    def construct(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        if self.use_se and self.stride != 1:
            out = self.e2(out)
        else:
            out = self.conv2(out)
            out = self.bn2(out)
            out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.se_block:
            out_se = out
            out = self.se_global_pool(out, (2, 3))
            out = self.se_dense_0(out)
            out = self.relu(out)
            out = self.se_dense_1(out)
            out = self.se_sigmoid(out)
            out = F.reshape(out, F.shape(out) + (1, 1))
            out = self.se_mul(out, out_se)

        if self.down_sample:
            identity = self.down_sample_layer(identity)

        out = out + identity
        out = self.relu(out)

        return out

In [19]:
class ResidualBlockBase(nn.Cell):
    def __init__(self,
                 in_channel,
                 out_channel,
                 stride=1,
                 use_se=False,
                 se_block=False,
                 res_base=True):
        super(ResidualBlockBase, self).__init__()
        self.res_base = res_base
        self.conv1 = _conv3x3(in_channel, out_channel, stride=stride, res_base=self.res_base)
        self.bn1d = _bn(out_channel)
        self.conv2 = _conv3x3(out_channel, out_channel, stride=1, res_base=self.res_base)
        self.bn2d = _bn(out_channel)
        self.relu = nn.ReLU()

        self.down_sample = False
        if stride != 1 or in_channel != out_channel:
            self.down_sample = True

        self.down_sample_layer = None
        if self.down_sample:
            self.down_sample_layer = nn.SequentialCell([_conv1x1(in_channel, out_channel, stride,
                                                                 use_se=use_se, res_base=self.res_base),
                                                        _bn(out_channel, res_base)])

    def construct(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1d(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2d(out)

        if self.down_sample:
            identity = self.down_sample_layer(identity)

        out = out + identity
        out = self.relu(out)

        return out

构建ResNet50架构:

In [20]:
class ResNet(nn.Cell):
    def __init__(self,
                 block,
                 layer_nums,
                 in_channels,
                 out_channels,
                 strides,
                 num_classes,
                 use_se=False,
                 res_base=False):
        super(ResNet, self).__init__()

        if not len(layer_nums) == len(in_channels) == len(out_channels) == 4:
            raise ValueError("the length of layer_num, in_channels, out_channels list must be 4!")
        self.use_se = use_se
        self.res_base = res_base
        self.se_block = False
        if self.use_se:
            self.se_block = True

        if self.use_se:
            self.conv1_0 = _conv3x3(3, 32, stride=2, use_se=self.use_se)
            self.bn1_0 = _bn(32)
            self.conv1_1 = _conv3x3(32, 32, stride=1, use_se=self.use_se)
            self.bn1_1 = _bn(32)
            self.conv1_2 = _conv3x3(32, 64, stride=1, use_se=self.use_se)
        else:
            self.conv1 = _conv7x7(3, 64, stride=2, res_base=self.res_base)
        self.bn1 = _bn(64, self.res_base)
        self.relu = P.ReLU()

        if self.res_base:
            self.pad = nn.Pad(paddings=((0, 0), (0, 0), (1, 1), (1, 1)))
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode="valid")
        else:
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode="same")

        self.layer1 = self._make_layer(block,
                                       layer_nums[0],
                                       in_channel=in_channels[0],
                                       out_channel=out_channels[0],
                                       stride=strides[0],
                                       use_se=self.use_se)
        self.layer2 = self._make_layer(block,
                                       layer_nums[1],
                                       in_channel=in_channels[1],
                                       out_channel=out_channels[1],
                                       stride=strides[1],
                                       use_se=self.use_se)
        self.layer3 = self._make_layer(block,
                                       layer_nums[2],
                                       in_channel=in_channels[2],
                                       out_channel=out_channels[2],
                                       stride=strides[2],
                                       use_se=self.use_se,
                                       se_block=self.se_block)
        self.layer4 = self._make_layer(block,
                                       layer_nums[3],
                                       in_channel=in_channels[3],
                                       out_channel=out_channels[3],
                                       stride=strides[3],
                                       use_se=self.use_se,
                                       se_block=self.se_block)

        self.mean = P.ReduceMean(keep_dims=True)
        self.flatten = nn.Flatten()
        self.end_point = _fc(out_channels[3], num_classes, use_se=self.use_se)

    def _make_layer(self, block, layer_num, in_channel, out_channel, stride, use_se=False, se_block=False):
        layers = []

        resnet_block = block(in_channel, out_channel, stride=stride, use_se=use_se)
        layers.append(resnet_block)
        if se_block:
            for _ in range(1, layer_num - 1):
                resnet_block = block(out_channel, out_channel, stride=1, use_se=use_se)
                layers.append(resnet_block)
            resnet_block = block(out_channel, out_channel, stride=1, use_se=use_se, se_block=se_block)
            layers.append(resnet_block)
        else:
            for _ in range(1, layer_num):
                resnet_block = block(out_channel, out_channel, stride=1, use_se=use_se)
                layers.append(resnet_block)
        return nn.SequentialCell(layers)

    def construct(self, x):
        if self.use_se:
            x = self.conv1_0(x)
            x = self.bn1_0(x)
            x = self.relu(x)
            x = self.conv1_1(x)
            x = self.bn1_1(x)
            x = self.relu(x)
            x = self.conv1_2(x)
        else:
            x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if self.res_base:
            x = self.pad(x)
        c1 = self.maxpool(x)

        c2 = self.layer1(c1)
        c3 = self.layer2(c2)
        c4 = self.layer3(c3)
        c5 = self.layer4(c4)

        out = self.mean(c5, (2, 3))
        out = self.flatten(out)
        out = self.end_point(out)

        return out

初始化训练集：

In [40]:
train_dataset = create_dataset(train_dir, batch_size=50, repeat_num=1)

初始化网络：

In [41]:
net = ResNet(ResidualBlock,
                  [3, 4, 6, 3],
                  [64, 256, 512, 1024],
                  [256, 512, 1024, 2048],
                  [1, 2, 2, 2],
                  10)

定义损失函数：

In [42]:
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

定义优化器：

In [43]:
optim = nn.Momentum(params=net.trainable_params(),
                    learning_rate=lr,
                    momentum=momentum)

初始化网络模型：

In [44]:
model = ms.Model(network=net, loss_fn=loss, optimizer=optim, metrics={"Accuracy": nn.Accuracy()})

模型训练：

In [45]:
model.train(train_dataset=train_dataset, epoch=epochs,
            callbacks=[LossMonitor(lr_init=lr, per_print_times=100)])

Epoch time: 77300.139 ms, per step time: 858.890 ms, avg loss: 1.984
Epoch:[  1/ 20], step:[   90/   90], loss:[1.740/1.740], time:6317.800 ms, lr:0.01000
Epoch time: 6325.423 ms, per step time: 70.282 ms, avg loss: 1.740
Epoch time: 6313.465 ms, per step time: 70.150 ms, avg loss: 1.802
Epoch:[  3/ 20], step:[   90/   90], loss:[1.628/1.628], time:6312.927 ms, lr:0.01000
Epoch time: 6318.117 ms, per step time: 70.201 ms, avg loss: 1.628
Epoch time: 6324.910 ms, per step time: 70.277 ms, avg loss: 1.689
Epoch:[  5/ 20], step:[   90/   90], loss:[1.633/1.633], time:6307.847 ms, lr:0.01000
Epoch time: 6312.997 ms, per step time: 70.144 ms, avg loss: 1.633
Epoch time: 6327.607 ms, per step time: 70.307 ms, avg loss: 1.427
Epoch:[  7/ 20], step:[   90/   90], loss:[1.466/1.466], time:6313.949 ms, lr:0.01000
Epoch time: 6320.307 ms, per step time: 70.226 ms, avg loss: 1.466
Epoch time: 6313.997 ms, per step time: 70.156 ms, avg loss: 1.249
Epoch:[  9/ 20], step:[   90/   90], loss:[1.209/1.

保存模型

In [46]:
ms.save_checkpoint(net, "resnet50.ckpt")

# 推理

处理测试图片

In [21]:
def normalize(image):
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]
    image = cv2.resize(image, (224, 224), cv2.INTER_LINEAR)
    image = image / 1.0
    image = (image[:, :] - mean) / std
    image = image[:, :, ::-1].transpose((2, 0, 1))  # HWC-->CHW
    return image

迭代处理

In [22]:
def pre_deal(data_path):
    image = cv2.imread(data_path)
    norm_img = normalize(image)
    images = [norm_img]
    images = ms.Tensor(images, ms.float32)
    return images

推理函数

In [23]:
def infer(ckpt_path, data_path):
    image = pre_deal(data_path)

    net = ResNet(ResidualBlock,
                  [3, 4, 6, 3],
                  [64, 256, 512, 1024],
                  [256, 512, 1024, 2048],
                  [1, 2, 2, 2],
                  10)
    param_dict = ms.load_checkpoint(ckpt_path)
    ms.load_param_into_net(net, param_dict)
    loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    model = ms.Model(net, loss, metrics={"Accuracy": nn.Accuracy()})

    output = model.predict(image)
    pred = argmax(output, axis=1)
    return pred

保存推理结果

In [24]:
ckpt_path = 'resnet50.ckpt'

# 存储结果
pred_list = []

# 提取图片顺序
key = lambda x: int(x[4:-4])

def info(test_dir):
    """推理图片"""
    path_list = os.listdir(os.path.join(test_dir))
    path_list.sort(key=key)
    for path in path_list:
        path = os.path.join(test_dir) + '/' + path
        print(path)
        result = infer(ckpt_path, path)
        print("The class is", result[0])
        result = str(result[0])
        result = result + "\n"
        pred_list.append(result)

# 粗糙地手动分批
test_dir1 = "dataset/test/part1"
test_dir2 = "dataset/test/part2"
test_dir3 = "dataset/test/part3"
test_dir4 = "dataset/test/part4"
test_dir5 = "dataset/test/part5"

In [24]:
info(test_dir1)

dataset/test/part1/img_0.jpg
The class is 1
dataset/test/part1/img_1.jpg
The class is 3
dataset/test/part1/img_2.jpg
The class is 0
dataset/test/part1/img_3.jpg
The class is 5
dataset/test/part1/img_4.jpg
The class is 0
dataset/test/part1/img_5.jpg
The class is 8
dataset/test/part1/img_6.jpg
The class is 4
dataset/test/part1/img_7.jpg
The class is 4
dataset/test/part1/img_8.jpg
The class is 9
dataset/test/part1/img_9.jpg
The class is 1
dataset/test/part1/img_10.jpg
The class is 1
dataset/test/part1/img_11.jpg
The class is 4
dataset/test/part1/img_12.jpg
The class is 9
dataset/test/part1/img_13.jpg
The class is 1
dataset/test/part1/img_14.jpg
The class is 7
dataset/test/part1/img_15.jpg
The class is 5
dataset/test/part1/img_16.jpg
The class is 7
dataset/test/part1/img_17.jpg
The class is 5
dataset/test/part1/img_18.jpg
The class is 3
dataset/test/part1/img_19.jpg
The class is 9
dataset/test/part1/img_20.jpg
The class is 2
dataset/test/part1/img_21.jpg
The class is 1
dataset/test/part1/i

In [25]:
info(test_dir2)

dataset/test/part2/img_100.jpg
The class is 9
dataset/test/part2/img_101.jpg
The class is 5
dataset/test/part2/img_102.jpg
The class is 7
dataset/test/part2/img_103.jpg
The class is 4
dataset/test/part2/img_104.jpg
The class is 5
dataset/test/part2/img_105.jpg
The class is 2
dataset/test/part2/img_106.jpg
The class is 4
dataset/test/part2/img_107.jpg
The class is 2
dataset/test/part2/img_108.jpg
The class is 9
dataset/test/part2/img_109.jpg
The class is 3
dataset/test/part2/img_110.jpg
The class is 9
dataset/test/part2/img_111.jpg
The class is 0
dataset/test/part2/img_112.jpg
The class is 1
dataset/test/part2/img_113.jpg
The class is 2
dataset/test/part2/img_114.jpg
The class is 7
dataset/test/part2/img_115.jpg
The class is 1
dataset/test/part2/img_116.jpg
The class is 4
dataset/test/part2/img_117.jpg
The class is 4
dataset/test/part2/img_118.jpg
The class is 0
dataset/test/part2/img_119.jpg
The class is 4
dataset/test/part2/img_120.jpg
The class is 1
dataset/test/part2/img_121.jpg
The

In [26]:
info(test_dir3)

dataset/test/part3/img_200.jpg
The class is 1
dataset/test/part3/img_201.jpg
The class is 7
dataset/test/part3/img_202.jpg
The class is 5
dataset/test/part3/img_203.jpg
The class is 2
dataset/test/part3/img_204.jpg
The class is 4
dataset/test/part3/img_205.jpg
The class is 5
dataset/test/part3/img_206.jpg
The class is 4
dataset/test/part3/img_207.jpg
The class is 4
dataset/test/part3/img_208.jpg
The class is 4
dataset/test/part3/img_209.jpg
The class is 7
dataset/test/part3/img_210.jpg
The class is 5
dataset/test/part3/img_211.jpg
The class is 9
dataset/test/part3/img_212.jpg
The class is 9
dataset/test/part3/img_213.jpg
The class is 2
dataset/test/part3/img_214.jpg
The class is 4
dataset/test/part3/img_215.jpg
The class is 0
dataset/test/part3/img_216.jpg
The class is 2
dataset/test/part3/img_217.jpg
The class is 1
dataset/test/part3/img_218.jpg
The class is 9
dataset/test/part3/img_219.jpg
The class is 7
dataset/test/part3/img_220.jpg
The class is 0
dataset/test/part3/img_221.jpg
The

In [27]:
file = open("pred.txt", "a")
file.writelines(pred_list)
file.close()

In [28]:
pred_list = []

In [25]:
info(test_dir4)

dataset/test/part4/img_300.jpg
The class is 5
dataset/test/part4/img_301.jpg
The class is 4
dataset/test/part4/img_302.jpg
The class is 8
dataset/test/part4/img_303.jpg
The class is 5
dataset/test/part4/img_304.jpg
The class is 2
dataset/test/part4/img_305.jpg
The class is 0
dataset/test/part4/img_306.jpg
The class is 1
dataset/test/part4/img_307.jpg
The class is 9
dataset/test/part4/img_308.jpg
The class is 0
dataset/test/part4/img_309.jpg
The class is 4
dataset/test/part4/img_310.jpg
The class is 4
dataset/test/part4/img_311.jpg
The class is 9
dataset/test/part4/img_312.jpg
The class is 0
dataset/test/part4/img_313.jpg
The class is 8
dataset/test/part4/img_314.jpg
The class is 1
dataset/test/part4/img_315.jpg
The class is 1
dataset/test/part4/img_316.jpg
The class is 1
dataset/test/part4/img_317.jpg
The class is 9
dataset/test/part4/img_318.jpg
The class is 5
dataset/test/part4/img_319.jpg
The class is 1
dataset/test/part4/img_320.jpg
The class is 9
dataset/test/part4/img_321.jpg
The

In [26]:
info(test_dir5)

dataset/test/part5/img_400.jpg
The class is 3
dataset/test/part5/img_401.jpg
The class is 1
dataset/test/part5/img_402.jpg
The class is 9
dataset/test/part5/img_403.jpg
The class is 5
dataset/test/part5/img_404.jpg
The class is 8
dataset/test/part5/img_405.jpg
The class is 4
dataset/test/part5/img_406.jpg
The class is 5
dataset/test/part5/img_407.jpg
The class is 4
dataset/test/part5/img_408.jpg
The class is 8
dataset/test/part5/img_409.jpg
The class is 8
dataset/test/part5/img_410.jpg
The class is 7
dataset/test/part5/img_411.jpg
The class is 3
dataset/test/part5/img_412.jpg
The class is 0
dataset/test/part5/img_413.jpg
The class is 3
dataset/test/part5/img_414.jpg
The class is 7
dataset/test/part5/img_415.jpg
The class is 9
dataset/test/part5/img_416.jpg
The class is 1
dataset/test/part5/img_417.jpg
The class is 0
dataset/test/part5/img_418.jpg
The class is 3
dataset/test/part5/img_419.jpg
The class is 1
dataset/test/part5/img_420.jpg
The class is 5
dataset/test/part5/img_421.jpg
The

In [27]:
file = open("pred.txt", "a")
file.writelines(pred_list)
file.close()